In [34]:
import os
import random
import sklearn
import pandas as pd
import numpy as np

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

2021-12-29 13:19:03.160508: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/z/GWT/gap_sdk/install/workstation/lib
2021-12-29 13:19:03.160525: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [87]:
# path
csv_path = "/home/z/PycharmProjects/gap_model/in/_annotations.csv"
data_path = "./in"
df = pd.read_csv(csv_path)
df = df[['filename', 'xmin', 'ymin', 'xmax', 'xmax']]
print(df)

                                                filename  xmin  ymin  xmax  \
0      acqui4_imag2_3850_png.rf.1f46d8e1abc87614273d1...    46    53    64   
1      acqui4_imag2_3850_png.rf.1f46d8e1abc87614273d1...     8     0    29   
2      acqui4_imag2_1474_png.rf.1f46cccce5aa89dd498cc...    10    33    29   
3      acqui4_imag2_1474_png.rf.1f46cccce5aa89dd498cc...    50    37    73   
4      acqui5_image_525_png.rf.1fa445144d951bacf962d1...    46    17    66   
...                                                  ...   ...   ...   ...   
11743  acqui2_image_1185_png.rf.be42d1e782b4ed649c6b5...    59    17    76   
11744  acqui2_image_1185_png.rf.be42d1e782b4ed649c6b5...    48    30    64   
11745  acqui2_image_1185_png.rf.be42d1e782b4ed649c6b5...    10    33    28   
11746  acqui2_image_1185_png.rf.be42d1e782b4ed649c6b5...    36     2    47   
11747  acqui2_image_1185_png.rf.be42d1e782b4ed649c6b5...    48     0    60   

       xmax  
0        64  
1        29  
2        29  
3      

In [32]:
# data shuffle
df_shuffle = df.sample(frac=1, random_state=1).reset_index(drop=True)
print(df_shuffle)

                                                filename  xmin  ymin  xmax  \
0      acqui1_im_110_png.rf.d4d033b475630f4cbd95898e3...     0    20    30   
1      acqui3_imag_2235_png.rf.07b0f9cf3a533cdb106ce0...     0    37    23   
2      acqui2_image_3867_png.rf.66a92fe4522016b8355cf...    61    37    79   
3      acqui2_image_121_png.rf.30250cf9cdc9183f806c92...    36     2    47   
4      acqui3_imag_1942_png.rf.1e769755ae18880ea5f2b1...    60    16    79   
...                                                  ...   ...   ...   ...   
11743  acqui4_imag2_3966_png.rf.09480dbb00223f7571156...     6     7    23   
11744  acqui3_imag_1279_png.rf.ac8a242359a435389d5d7c...    55    38    79   
11745  acqui2_image_2116_png.rf.32dac178d1815893db052...    12    23    26   
11746  acqui2_image_1296_png.rf.cebfe62cd845c2d117653...    26     0    35   
11747  acqui4_imag2_2063_png.rf.24307e2567d7d8e0b47bf...    44    23    69   

       xmax  
0        30  
1        23  
2        79  
3      

In [72]:
# train test split
split_index = round(len(df_shuffle.index) * 0.08)
print(split_index)
df_train = df_shuffle.iloc[split_index:]
df_test = df_shuffle.iloc[:split_index]
print(f"total : {len(df_shuffle.index)}")
print(f"train : {len(df_train.index)}")
print(f"test : {len(df_test.index)}")
print(f"df_train : \n{df_train}")
print(f"df_test : \n{df_test}")


940
total : 11748
train : 10808
test : 940
df_train : 
                                                filename  xmin  ymin  xmax  \
940    acqui5_image_247_png.rf.309368f9e4c069d786d3bb...     0    20    24   
941    acqui3_imag_1033_png.rf.f8b6e81c6ef53e81b5379f...    61    40    79   
942    acqui5_image_542_png.rf.1829b41daa9f0744365a5b...    12    23    32   
943    acqui2_image_3370_png.rf.a6e72f71040f9c8f5c1d0...    51    58    73   
944    acqui2_image_3465_png.rf.1670f551242ac81cf18da...    15    14    28   
...                                                  ...   ...   ...   ...   
11743  acqui4_imag2_3966_png.rf.09480dbb00223f7571156...     6     7    23   
11744  acqui3_imag_1279_png.rf.ac8a242359a435389d5d7c...    55    38    79   
11745  acqui2_image_2116_png.rf.32dac178d1815893db052...    12    23    26   
11746  acqui2_image_1296_png.rf.cebfe62cd845c2d117653...    26     0    35   
11747  acqui4_imag2_2063_png.rf.24307e2567d7d8e0b47bf...    44    23    69   

       x

In [89]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(csv_path)


IndexError: list index out of range

In [76]:
# make train,test dir
train_path = f"{data_path}/train"
test_path = f"{data_path}/test"
if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(test_path):
    os.makedirs(test_path)

In [77]:
# move copy all files to train. test dir
import shutil

for i in range(len(df_train.index)):
    target_train = df_train.iloc[i,0]
    shutil.copy(f"{data_path}/{target_train}", f"{train_path}/{target_train}")
    print("train copy finished")

for i in range(len(df_test.index)):
    target_test = df_test.iloc[i,0]
    shutil.copy(f"{data_path}/{target_test}", f"{test_path}/{target_test}")
    print("test copy finished")

In [80]:
# train,test df to csv
train_csv_path = f'{train_path}/train.csv'
test_csv_path = f'{test_path}/test.csv'
df_train.to_csv(train_csv_path, index=False)
df_test.to_csv(test_csv_path, index=False)

In [85]:
train_data = object_detector.DataLoader.from_csv(
    train_csv_path,
    train_path,
)
test_data = object_detector.DataLoader.from_csv(
    test_csv_path,
    test_path,
)
print(train_data)
print(test_data)

[None, None, None]
[None, None, None]


In [84]:
import csv

with open(train_csv_path, 'r') as train_csv:
    print(train_csv.read())

with open(test_csv_path, 'r') as test_csv:
    print(test_csv.read())

filename,xmin,ymin,xmax,xmax
acqui5_image_247_png.rf.309368f9e4c069d786d3bb2299cb0d71.jpg,0,20,24,24
acqui3_imag_1033_png.rf.f8b6e81c6ef53e81b5379f2646561633.jpg,61,40,79,79
acqui5_image_542_png.rf.1829b41daa9f0744365a5b7220db117a.jpg,12,23,32,32
acqui2_image_3370_png.rf.a6e72f71040f9c8f5c1d063189b13148.jpg,51,58,73,73
acqui2_image_3465_png.rf.1670f551242ac81cf18da092a1d031bb.jpg,15,14,28,28
acqui2_image_1214_png.rf.aafe9b31012bd73fd5781aa8fa69a7e9.jpg,55,12,74,74
acqui5_image_246_png.rf.44490aacd5f6247c9785b933a58e72cf.jpg,0,20,24,24
acqui1_im_18_png.rf.dc08ec2c5a9ffaaeb96648db3dcfed41.jpg,7,20,21,21
acqui4_imag2_1310_png.rf.4da21b5fbe43d2f1c48c693e8293fe26.jpg,11,2,27,27
acqui2_image_3867_png.rf.66a92fe4522016b8355cf1ef1a47d645.jpg,44,48,62,62
acqui2_image_1476_png.rf.a19cb482098ea7a514c88822c8b29ce0.jpg,35,3,49,49
acqui4_imag2_1173_png.rf.e41c1d6fe13c56dd3cf539b77691d25e.jpg,3,11,24,24
acqui4_imag2_2266_png.rf.2dae5310a64d92dfb514bb5df508c44f.jpg,1,16,24,24
acqui2_image_2588_png.rf.